# Exercises Week 9

**In this sequence of exercises we will use [networkx](https://networkx.github.io/) to simplify some graph function and check the correctness of some implmentation** 

**Note that Networkx works well on small graphs / networks, for larger networks it is better to use libraries like [Graph-tool](https://graph-tool.skewed.de/) which are based on C implementations** 

If you don't have Networkx installed please run the code below.

In [ ]:
import sys
!{sys.executable} -m pip install networkx

## Exercise 1 - play with graphs

Networkx allows some simple operation in graphs, for instance one can load a graph and plot it. However the layout will change every-time randomly as the nodes will be distributed using some randomized algorithm. 

```python
G = nx.karate_club_graph()
nx.draw(G, with_labels=True)
plt.show()
```

If you are interested to know from where this graph comes from, look at [this](https://en.wikipedia.org/wiki/Zachary%27s_karate_club) page.

In [1]:
%matplotlib inline

import networkx as nx
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np

fixed_positions = {0:(10.74,4.07),1:(9.76,6.48),2:(8.39,5.21),3:(10.37,1.98),4:(12.30,5.61),5:(13.31,3.28),6:(13.28,5.00),7:(8.41,7.06),8:(6.72,4.31),9:(5.77,1.38),10:(12.30,2.72),11:(12.75,4.05),12:(11.32,2.41),13:(8.70,2.88),14:(3.33,0.63),15:(1.88,2.01),16:(13.92,4.05),17:(10.77,5.61),18:(0.69,6.40),19:(9.05,1.38),20:(0.34,4.63),21:(11.56,6.22),22:(5.24,0.34),23:(1.88,7.49),24:(5.11,6.80),25:(4.31,8.52),26:(2.14,0.32),27:(3.65,6.64),28:(6.03,5.24),29:(0.77,2.91),30:(7.01,2.43),31:(6.61,7.86),32:(4.60,4.52),33:(4.39,2.91)}

def plot_karate(G, pr = []): 
    if len(pr) :
        nx.draw(G, with_labels=True, pos=fixed_positions, cmap=plt.get_cmap('RdYlBu'), node_color=pr)#node_color=np.array(np.around(kphi[:,i],decimals=1)))
    else : 
        nx.draw(G, with_labels=True, pos=fixed_positions)
    

G = nx.karate_club_graph()
nx.draw(G, with_labels=True)
plt.show()
plot_karate(G)

It is also easy to manipulate the graph by adding and removing edges or coloring in a different way. 

In [2]:
G.add_edge(2,4)
nx.draw(G, with_labels=True)

As well as computing the PageRank of nodes, clustering nodes, and perform some algorithmic transformation.

```python
nx.pagerank(G, alpha=0.85)
```

In [3]:
pr = nx.pagerank(G, alpha=0.85)
print(pr)
pr_vec = np.fromiter(pr.values(), dtype=float)
plot_karate(G, 1 - pr_vec)

An important matrix in graphs is the adjacency matrix which gives you structual information about the *connectivity* of the graph. In particular, every row of the adjacency matrix tells you to which other nodes a node is connected. 

In [4]:
plt.set_cmap(plt.get_cmap('Blues'))

A = nx.adjacency_matrix(G)
print(A.todense())
plt.matshow(A.todense())
plt.show()

Plotted that way the adjacecny matrix doesn't tell you much. However, we can reorder nodes and edges. 
**This part of the code used some advanced method you don't need to understand.**

If ordered proberly we should be able to see the two groups of nodes of the  graph. 

In [5]:
from sklearn.cluster import SpectralClustering

A = nx.adjacency_matrix(G)


#### DO NOT TRY TO UNDERSTAND 
# The method finds communities in a graph, I just use it to reorder the adjacency matrix. 
def reorder_adj_matrix(A, num_clust = 2) : 
    sc = SpectralClustering(num_clust, affinity='precomputed', n_init=100)
    sc.fit(A)
    clust = sc.labels_

    perm = np.argsort(clust)
    perm_mat = np.zeros((len(perm), len(perm)))

    for idx, i in enumerate(perm):
        perm_mat[idx, i] = 1

    A_ord = np.dot(np.dot(perm_mat, A.todense()), perm_mat.T)    
    return A_ord, clust
#### DO NOT TRY TO UNDERSTAND


A_ord, clust = reorder_adj_matrix(A)
plt.matshow(A_ord)
plt.show()

print("We now see the two communities of nodes")
plot_karate(G, clust)

**Exploratory Task**: You can now play with the small graph, adding and removing edges and see the effect of different values of $\alpha$ (**note** that in some other notations the $\alpha$ is called $c$) in the PageRank. Also try Pagerank in different graphs. 

## Exercise 2 - Linear embeddings

This exercise is about Linear embeddings and how to perform dimensionality reduction on the adjacency matrix or a matrix derived from the adjacency matrix. 

Linear embeddings typically perform some linear dimensionality reduction on, variations of the Adjacency matrix $A\in \{0,1\}^{n \times n}$ for an **undirected graph** $G = (V,E)$. 

A linear embedding finds a matrix $Z \in \mathbb{R}^{n\times d}$, such that the i'th row of $Z$ is the $d$-dimensional representation of node (or vertex) d in the graph.

The methods for learning such an embedding matrix $Z$ try to minimize a "reconstruction error" of the embedding with respect to the adjacecy matrix
which is formalized in the following loss function $\mathcal{L}$.
$$
\mathcal{L}(A,Z) = \sum_{(i,j) \in E}(Z_{i}^\top Z_{j} - A_{i,j})^2 = \|ZZ^\top - A\|_F^2
$$

1. Show that if instead of learning one matrix Z, you are allowed to learn two matrices $B \in \mathbb{R}^{n\times d}$ and $C \in \mathbb{R}^{n\times d}$, the best solution to the above minimization can be obtained by SVD on $A$.


2. What is the gradient matrix of the above function for a single row $Z_{i\cdot}$ of the matrix $Z$?


3. Implement the linear embedding above using gradient descent and $\hat{A}$ the adjacency matrix


4. \[Optional\] Implement the same procedure using pytorch. 

In [ ]:
np.random.seed(1286214515)

def lin_grads(A, Z): 
    """ Compute gradient of loss relative relative to all entries in Z
    
        Args: 
            A: np.array shape n x n
            Z: np.array shape n x d
        
        Returns:
            res: np.array shape n x d
    """
    res = np.zeros(Z.shape)
    ### YOUR CODE HERE    
    ### END CODE
    return res

def lin_loss(A, Z): 
    """ Compute gradient (matrix) of derivatives of loss relative relative to all entries in Z
    
        Args: 
            A: np.array shape n x n
            Z: np.array shape n x d
            
        Returns:
            res: float scaler, 
    """
    res = 0
    ### YOUR CODE HERE    
    ### END CODE
    return res

    
def learn(A, dim, grad = lin_grads, loss = lin_loss, step_size=0.001, steps=1000):
    """ Gradient Descent learning Algorithm 
    
        Args:
            A: np.array shape n x n
            dim: int dimension of embedding
            grad: function - gradient computation function
            loss: function - loss computation function
            step_size: float - learning rate for gradient descent
            steps: int - number of iterations of gradient descent

        Returns:
            Z: np.array n x d - the learned parameters                
    """
    Z = np.random.rand(A.shape[0], dim)
    i = 0
    ### YOUR CODE HERE    
    ### END CODE
    return Z

# Project the graph in 2D and plot it as points in the space
Z = learn(A, 2)

# As colors to see what happens we can use the communities 
# that have been found with the magic method in the beginning.
# A good embedding method should be able to separate the two colors! 
colors = ['red' if c == 0 else 'blue' for c in clust]
plt.scatter(np.array(Z[:,0]), np.array(Z[:,1]), c=colors)
plt.show()

# Exercise 3 - Random walks and PageRank

Many embedding methods rely on random walks to capture the structure of the neighbors of a node. Random walks are tightly connected to the Pagerank as a limit of convergence in performing random walks on a graph. 

In this theoretical exercise we will reason about PageRank and its properties. Remember that the Personalized Pagerank in a undirected graph can be experssed by the equation 
$$
\mathbf{p} = \alpha W\mathbf{p} - (1-\alpha)\mathbf{r} 
$$
where $W = A^\top D^{-1}$ is the transition matrix, $\mathbf{r}$ is called the restart vector and represents the starting nodes of the random walk where the walker will jump back with probability $(1-\alpha)$. 

To compute the Pagerank vector one could 

Let us assume $\alpha = 0.85$ and $\mathbf{r} = \frac{1}{n}\mathbf{1}_n$ is uniformly distributed where $\mathbf{1}_n$ is the $n$-dimensional vectors with all 1s. 

**Optional**: Prove that $A^\top D^{-1}$ corresponds to the transition matrix.


1. What is the PageRank of a complete graph of n nodes? 


2. What is the PageRank of a complete bipartite graph with 2 nodes on one side and 4 on the other side and $\alpha=1$? 

 
3. A path graph is a graph where each node is connected to the consecutive. What is the PageRank with 3 nodes when $\alpha=1$? 
**Hint**: Remember that the elements in the PageRank vector sum to 1 (it's a distribution over nodes). 


4. Provide an implementation of the Power Iteration method  in the cell below

5. Check your code, why does the method fails for the path graph and $\alpha = 1$?



In [7]:
from networkx.algorithms import bipartite

# Tolerance is the sum of the absolute differences among the pagerank vectors in consequent iterations
# The iteration stops is the the differences are less than the tolerance
def my_pagerank(G, alpha = 0.85, r = None, max_iter = 100, tol=1e-06) : 
    """ Compute pagerank with power method
    
        Args:
            G: networkx graph
            alpha: float, Page rank parameters
            r: np.array shape n,
            max_iter: int - max iteration of power method
            tol: float - stopping criterion tolerance. 
        Returns
            p: np.array n, - the page rank of each node
    """
    A = nx.adjacency_matrix(G)
    n = G.number_of_nodes()
    p = np.full(n, 1/n)

    if r is None : 
        r = np.full(n, 1/n)

    ### YOUR CODE HERE
    ### END CODE
    return p

Gb = nx.complete_bipartite_graph(4, 5, create_using=None)
print('Pagerank bipartite')
print(nx.pagerank_numpy(Gb, alpha=1))
print(my_pagerank(Gb))


Gc = nx.complete_graph(5)
print('Pagerank complete')
print(nx.pagerank(Gc, alpha=0.85))
print(my_pagerank(Gc))

Gl = nx.path_graph(3)
print('Pagerank path graph')
print(nx.pagerank_numpy(Gl, alpha=1))
print(my_pagerank(Gl, alpha=1))

## Exercise 4 - VERSE

In this exercise, we will try to implement and run one network embedding introduced in the lecture, called [VERSE](https://arxiv.org/pdf/1803.04742.pdf).

VERSE takes as input a graph $G=(V,E)$, a row-normalized similarity matrix $S$ and computes an embedding matrix $Z \in \mathbb{R}^{n\times d}$ by  minimizing the following cross-entropy loss between the rows in $S$ and the rows in $ZZ^\top$.

The VERSE loss function for each node $i$ is 
$$
\mathcal{L}(S_i,Z) = -S_i \log(\text{softmax}(Z_iZ^\top)) 
$$

which gives a total global loss over all nodes as 
$$
\mathcal{L}(S,Z) = \sum_{i \in V}\mathcal{L}(S_i,Z)
$$
here $\text{softmax}(\cdot)$ is the [softmax](https://en.wikipedia.org/wiki/Softmax_function) function. 

The similarity matrix in case of VERSE is typically the Personalized PageRank (PPR) matrix, which is obtained row-by-row running the PageRank algorithm with $\alpha = 0.85$.

The PPR matrix is an $n \times n$ matrix where the i'th row is the Page rank vector given by the restart vector that is the indicator vector for node i i.e. a vector with all zeros except a one in position i.


We will first implement the full VERSE method and understand how it works in practice 
1. Compute the Personalized PageRank matrix starting the random walk computation from all the nodes individually using the formula above or iterating with my_pagerank
2. Implement the VERSE Loss function in verse_loss below

3. Compute the gradient of the loss function and implement it in verse_grads. If you cannot come up with the gradient you can instead use pytorch in the following cell, where you must implement the loss in pytorch 
**Hint: do it row-wise (i.e., node-by-node) and remember the gradient of cross-entropy with softmax activation**


3. Try to change the parameters and use another similarity function. What other similarity function among nodes can you use? 

In [66]:
from scipy.special import softmax

def pagerank_matrix(G, alpha = 0.85):
    """ Compute the personalized Page Rank Matrix of G
        
        Args:
            G: networkx graph
            alpha: float
    
        returns
        np.array number_of_nodes(G) x number_of_nodes(G)
    """
    n = G.number_of_nodes()
    A = nx.adjacency_matrix(G).todense()    
    ### YOUR CODE HERE
    ### END CODE

P = pagerank_matrix(G)
n = G.number_of_nodes()

# Sanity checks on the matrix
per = np.zeros(n)
per[1] = 1
print(my_pagerank(G,r=per))
print (P[1, :])

print(P.sum(axis=1)) # Every row should sum to 1.


def verse_loss(S, Z): 
    """ Compute the loss of VERSE 
    
        Args:
            S: np.array shape n x n
            Z: np.array shape n x d
            
        Returns:
            loss: float    
    """
    loss = 0
    ### YOUR CODE HERE
    ### END CODE
    return loss

verse_loss(.5 *np.ones((2,2)),np.array([[1,2],[3,4]])) 

def verse_grads(S, Z): 
    """ Compute the loss of VERSE 
    
        Args:
            S: np.array shape n x n
            Z: np.array shape n x d
            
        Returns:
            np.array shape n x d    
    """

    grads = np.zeros(Z.shape)
    ### YOUR CODE HERE 
    ### END CODE
    return grads

Z = learn(P, 2, grad=verse_grads, loss=verse_loss, step_size = 0.005, steps=4000)
fig, ax = plt.subplots(1, 1, figsize=(12,10))
ax.scatter(Z[:,0], Z[:,1], c=colors)
ax.set_title('Verse Embedding: Colors are Cluster colors from earlier', fontsize=20)
plt.show()


In [68]:
import torch
import torch.nn
from torch import optim


def t_verse_loss(S, Z):
    """ Verse Loss function in pytorch 
        
        torch.nn.functional.log_softmax may come in handy
        
        Args:
            S: torch.tensor shape (n x n)
            Z: torch.tensor shape (n x d)
    
        Returns:
            loss: float scalar            
    """
    loss = 0
    n = S.shape[0]
    ### YOUR CODE HERE
    ### END CODE
    return loss

def t_fit(target_matrix, d, epochs, lr):
    """ Pytorch gradient descent 
    
        Args:
            target_matrix: torch.tensor shape (n x n), the similary matrix  to approximate
            d: int, positive embedding dimension
            eposh: int, number of iterations to run
            lr: float - learning rate of Gradient Descent
    
        Returns
            W: torch.tensor shape (n x d) - the learned embedding
    """
    n = target_matrix.shape[0]
    W = torch.rand(n, d, requires_grad=True)
    optimizer = optim.SGD(params={W}, lr=lr)
    for i in range(epochs):
        optimizer.zero_grad()
        loss = t_verse_loss(target_matrix, W)
        loss.backward()
        if i % 20 == 0:
            print(f'epoch {i}: verse loss: {loss.item()}\r', end='')
        optimizer.step()
    print('\nfinal loss', loss.item())
    return W.detach()
torch_P = torch.from_numpy(P).float()
Z = t_fit(target_matrix=torch_P, d=2, epochs=4000, lr=0.005).numpy()
fig, ax = plt.subplots(1, 1, figsize=(12,10))
ax.scatter(Z[:,0], Z[:,1], c=colors)
ax.set_title('Verse Embedding: Colors are Cluster colors from earlier', fontsize=20)
plt.show()
    